In [ ]:
## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,89.55,23,0,7.85,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,48.20,45,75,17.22,broken clouds
2,2,Bluff,NZ,-46.60,168.33,57.99,91,100,11.01,overcast clouds
3,3,Cortez,US,37.35,-108.59,33.80,55,1,10.29,clear sky
4,4,Mulege,MX,26.88,-111.98,59.29,61,20,2.93,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,89.55,23,0,7.85,clear sky
8,8,Port Elizabeth,ZA,-33.92,25.57,71.60,88,75,17.22,broken clouds
12,12,Rikitea,PF,-23.12,-134.97,77.86,67,6,6.33,clear sky
14,14,Dubbo,AU,-32.25,148.62,80.60,34,100,8.05,overcast clouds
15,15,Launceston,AU,-41.45,147.17,73.00,37,40,17.22,scattered clouds
20,20,Kapaa,US,22.08,-159.32,73.40,88,90,7.56,overcast clouds
22,22,Inirida,CO,3.87,-67.92,72.97,98,92,1.19,light rain
23,23,Pedernales,DO,18.04,-71.74,76.26,64,0,2.42,clear sky
28,28,Port Alfred,ZA,-33.59,26.89,71.01,94,13,7.00,few clouds
29,29,Puerto Ayora,EC,-0.74,-90.35,71.01,94,8,6.73,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                252
City                   252
Country                249
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = float("NaN")
clean_df.replace("", nan_value, inplace = True)
clean_df.dropna(subset = ["Country"], inplace = True)
clean_df

C:\Users\fl893222\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4385: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\fl893222\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,89.55,23,0,7.85,clear sky
8,8,Port Elizabeth,ZA,-33.92,25.57,71.60,88,75,17.22,broken clouds
12,12,Rikitea,PF,-23.12,-134.97,77.86,67,6,6.33,clear sky
14,14,Dubbo,AU,-32.25,148.62,80.60,34,100,8.05,overcast clouds
15,15,Launceston,AU,-41.45,147.17,73.00,37,40,17.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Sur,OM,22.57,59.53,74.52,51,0,12.66,clear sky
691,691,Tautira,PF,-17.73,-149.15,75.20,88,75,11.41,light rain
692,692,Voi,KE,-3.39,38.56,77.31,74,50,4.83,scattered clouds
697,697,San Andres,CO,12.58,-81.70,82.40,78,75,14.99,light rain


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,89.55,clear sky,-33.65,115.33,
8,Port Elizabeth,ZA,71.60,broken clouds,-33.92,25.57,
12,Rikitea,PF,77.86,clear sky,-23.12,-134.97,
14,Dubbo,AU,80.60,overcast clouds,-32.25,148.62,
15,Launceston,AU,73.00,scattered clouds,-41.45,147.17,
...,...,...,...,...,...,...,...
690,Sur,OM,74.52,clear sky,22.57,59.53,
691,Tautira,PF,75.20,light rain,-17.73,-149.15,
692,Voi,KE,77.31,scattered clouds,-3.39,38.56,
697,San Andres,CO,82.40,light rain,12.58,-81.70,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping")
hotel_df

hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,89.55,clear sky,-33.65,115.33,Observatory Guest House
8,Port Elizabeth,ZA,71.60,broken clouds,-33.92,25.57,39 On Nile Guest House
12,Rikitea,PF,77.86,clear sky,-23.12,-134.97,Pension Maro'i
14,Dubbo,AU,80.60,overcast clouds,-32.25,148.62,Comfort Inn Blue Lagoon
15,Launceston,AU,73.00,scattered clouds,-41.45,147.17,Hotel Grand Chancellor Launceston
...,...,...,...,...,...,...,...
690,Sur,OM,74.52,clear sky,22.57,59.53,Sur Plaza Hotel
691,Tautira,PF,75.20,light rain,-17.73,-149.15,Pension Oaoa
692,Voi,KE,77.31,scattered clouds,-3.39,38.56,Voi Wildlife Lodge
697,San Andres,CO,82.40,light rain,12.58,-81.70,Hotel Arena Blanca


In [17]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,89.55,clear sky,-33.65,115.33,Observatory Guest House
8,Port Elizabeth,ZA,71.60,broken clouds,-33.92,25.57,39 On Nile Guest House
12,Rikitea,PF,77.86,clear sky,-23.12,-134.97,Pension Maro'i
14,Dubbo,AU,80.60,overcast clouds,-32.25,148.62,Comfort Inn Blue Lagoon
15,Launceston,AU,73.00,scattered clouds,-41.45,147.17,Hotel Grand Chancellor Launceston
...,...,...,...,...,...,...,...
690,Sur,OM,74.52,clear sky,22.57,59.53,Sur Plaza Hotel
691,Tautira,PF,75.20,light rain,-17.73,-149.15,Pension Oaoa
692,Voi,KE,77.31,scattered clouds,-3.39,38.56,Voi Wildlife Lodge
697,San Andres,CO,82.40,light rain,12.58,-81.70,Hotel Arena Blanca


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))